In [3]:
import os
from os import access
import xlwings as xw
from connect_to_dhan import Connection
from dhanhq import dhanhq, DhanContext
import numpy as np
import pandas as pd
from dhanhq import marketfeed
import yaml
import time
from dotenv import load_dotenv
# Load environment variables
load_dotenv(override=True)


True

In [5]:

client_id =  os.getenv('DHAN_CLIENT_ID')
access_token = os.getenv('DHAN_ACCESS_TOKEN')

# For development/testing
dhan_context = DhanContext(client_id, access_token, use_sandbox=True)
dhan_client = dhanhq(dhan_context)

dhan_client.get_holdings()


{'status': 'success', 'remarks': '', 'data': []}

In [4]:
k = [1,2,3,5]

In [5]:
pd.DataFrame(k)

,0
0,1
1,2
2,3
3,5
